<a href="https://colab.research.google.com/github/himanshudas13/MovieRecommender/blob/main/content_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DATA COLLECTION

In [ ]:
!pip install nltk
import nltk

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler



In [ ]:
movies = pd.read_csv('/content/tmdb_5000_credits.csv', delimiter=',')
credits = pd.read_csv('/content/tmdb_5000_movies.csv', delimiter=',')
# movies.head()
# credits.head()
movies=movies.merge(credits,on='title') #based on name as primary key merge 2 tables into one
# movies.head()

DATA PREPROCESSING

In [ ]:
# print(movies.columns)
# Drop specified columns if they exist
columns_to_drop = [
       'production_companies_x',
        'budget_y', 'genres_y',
       'keywords_y', 'overview_y', 'popularity_y', 'production_companies_y',
       'release_date_y', 'revenue_y', 'tagline_y','original_title', 'vote_count', 'runtime', 'status', 'tagline', 'spoken_languages', 'homepage', 'id', 'original_language', 'production_companies', 'production_countries']

# Drop only the columns that exist in the DataFrame
movies = movies.drop(columns=[col for col in columns_to_drop if col in movies.columns])
movies.columns


Index(['movie_id', 'title', 'cast', 'crew', 'budget', 'genres', 'keywords',
       'overview', 'popularity', 'release_date', 'revenue', 'vote_average'],
      dtype='object')

In [ ]:

# Step 3: Log-transform the 'revenue' column
movies['revenue'] = np.log1p(movies['revenue'])  # np.log1p is used to avoid log(0) issues
movies['budget'] = np.log1p(movies['budget'])  # np.log1p is used to avoid log(0) issues
movies['popularity'] = np.log1p(movies['popularity'])  # np.log1p is used to avoid log(0) issues
# Extract year from 'release_date' if it exists
if 'release_date' in movies.columns:
    movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
    movies['release_year'] = movies['release_date'].dt.year
    movies = movies.drop(columns=['release_date'])

# Step 4: Drop rows with missing values
movies = movies.dropna()

# Display the final DataFrame structure
# for year in movies['release_year']:
#   print(year)


In [ ]:
# prompt: print average value of budget popularity revenue and vote_average

# Calculate and print average values
average_budget = movies['budget'].mean()
average_popularity = movies['popularity'].mean()
average_revenue = movies['revenue'].mean()
average_vote_average = movies['vote_average'].mean()

print("Average Budget:", average_budget)
print("Average Popularity:", average_popularity)
print("Average Revenue:", average_revenue)
print("Average Vote Average:", average_vote_average)


Average Budget: 13.00348467283806
Average Popularity: 2.5185480106408242
Average Revenue: 12.22735615778338
Average Vote Average: 6.094526534859521


In [ ]:
# print(movies.head())
# movies.isnull().sum()
# movies.duplicated().sum()
movies.head()


,movie_id,title,cast,crew,budget,genres,keywords,overview,popularity,revenue,vote_average,release_year
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",19.283571,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",5.020174,21.748578,7.2,2009.0
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",19.519293,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",4.942232,20.683485,6.9,2007.0
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",19.316769,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,4.685614,20.596199,6.3,2015.0
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",19.336971,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,4.730153,20.804790,7.6,2012.0
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",19.376192,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",3.805039,19.464974,6.1,2012.0


In [ ]:
#details about the movie simplified
import ast

def get_first_three_actors(cast_str):
  actors = ast.literal_eval(cast_str)
  first_three_actors = [actor['name'] for actor in actors[:3]]
  return first_three_actors

def get_crew_members(crew_str):
  crew = ast.literal_eval(crew_str)
  crew_members = []
  for member in crew:
    if member['job'] in ['Director', 'Director of Photography', 'Cinematography', 'Producer', 'Writer', 'Screenwriter']:
      crew_members.append(member['name'])
  return crew_members

movies['genres'] = movies['genres'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)])
movies['keywords'] = movies['keywords'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)])
movies['cast'] = movies['cast'].apply(get_first_three_actors)
movies['crew'] = movies['crew'].apply(get_crew_members)
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [ ]:
for col in ['genres', 'cast', 'crew', 'overview', 'keywords']:
  movies[col] = movies[col].apply(lambda x: [i.replace(" ", "") for i in x])

In [ ]:

def combine_features(row):
  combined_features = []
  for feature in ['genres', 'cast', 'crew', 'overview', 'keywords']:
    combined_features.extend(row[feature])
  return combined_features

movies['combined_features'] = movies.apply(combine_features, axis=1)


In [ ]:

data_frame = movies.drop(columns=['genres', 'cast', 'crew', 'overview', 'keywords'])
data_frame['combined_features'] = data_frame['combined_features'].apply(lambda x: " ".join(x).lower())

data_frame.head()

,movie_id,title,budget,popularity,revenue,vote_average,release_year,combined_features
0,19995,Avatar,19.283571,5.020174,21.748578,7.2,2009.0,action adventure fantasy sciencefiction samwor...
1,285,Pirates of the Caribbean: At World's End,19.519293,4.942232,20.683485,6.9,2007.0,adventure fantasy action johnnydepp orlandoblo...
2,206647,Spectre,19.316769,4.685614,20.596199,6.3,2015.0,action adventure crime danielcraig christophwa...
3,49026,The Dark Knight Rises,19.336971,4.730153,20.804790,7.6,2012.0,action crime drama thriller christianbale mich...
4,49529,John Carter,19.376192,3.805039,19.464974,6.1,2012.0,action adventure sciencefiction taylorkitsch l...


MODEL BUILDING

In [ ]:

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

data_frame['combined_features'] = data_frame['combined_features'].apply(stem)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(data_frame['combined_features']).toarray()


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)


In [ ]:
# prompt: in the below code  generate a error movie not found if move not in dataframe

def recommend(movie):
  if movie not in data_frame['title'].values:
    return "Movie not found"  # Return an error message if the movie is not found

  i = data_frame[data_frame['title'] == movie].index[0]
  distances = sorted(list(enumerate(similarity[i])), reverse=True, key=lambda vector: vector[1])

  recommended_movies = []
  for j in distances[1:16]:
    recommended_movies.append(data_frame.iloc[j[0]].title)

  return recommended_movies


['Aliens',
 'Aliens vs Predator: Requiem',
 'Falcon Rising',
 'Terminator 2: Judgment Day',
 'Independence Day']

In [ ]:

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

# ... (preceding code remains the same)

# Extract numerical features for Euclidean distance
numerical_features = ['budget', 'popularity', 'revenue', 'release_year','vote_average']  # Add 'vote_average' if available
numerical_data = data_frame[numerical_features].fillna(0)  # Handle potential missing values

# Scale numerical features for fair comparison
# scaler = MinMaxScaler()
# scaled_numerical_data = scaler.fit_transform(numerical_data)

# Calculate Euclidean distances
euclidean_similarity = euclidean_distances(numerical_data)

def recommend_combined(movie):
  if movie not in data_frame['title'].values:
    return "Movie not found"

  i = data_frame[data_frame['title'] == movie].index[0]

  # Combine cosine similarity and Euclidean distance
  combined_similarity = (similarity[i] + (1 - euclidean_similarity[i])) / 2  # Simple averaging, adjust weights as needed

  distances = sorted(list(enumerate(combined_similarity)), reverse=True, key=lambda vector: vector[1])

  recommended_movies = []
  for j in distances[1:16]:
    recommended_movies.append(data_frame.iloc[j[0]].title)

  return recommended_movies




['Harry Potter and the Half-Blood Prince',
 'Up',
 'Alice in Wonderland',
 'Toy Story 3',
 'The Dark Knight']

In [ ]:
def final_recommendation(movie):
    # Get recommendations from the first method
    recommend1 = recommend(movie)

    # Get recommendations from the combined method
    recommend2 = recommend_combined(movie)

    # Find the common elements
    common_recommendations = recommend1+recommend2

    # Remove duplicates and return the list
    common_recommendations = list(set(common_recommendations))

    return list(common_recommendations)
final_recommendation('Avatar')

['Up',
 'Aliens',
 'Toy Story 3',
 'Aliens vs Predator: Requiem',
 'Alice in Wonderland',
 'Harry Potter and the Half-Blood Prince',
 'Terminator 2: Judgment Day',
 'The Dark Knight',
 'Falcon Rising',
 'Independence Day']